Knižnice a inicializácia SparkSwssion

In [39]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, when, count, mean
from pyspark.sql import functions as F
from pyspark.sql.types import NumericType

spark = SparkSession.builder.appName("zadanieTSVD").getOrCreate()

Načítanie dát

In [62]:
# header=True –> použije prvý riadok ako názvy stĺpcov
# inferSchema=True –> automaticky určí dátové typy
accidents = spark.read.csv("DATA/Accidents.csv",header=True,inferSchema=True)
casualties = spark.read.csv("DATA/Casualties.csv",header=True,inferSchema=True)
vehicles = spark.read.csv("DATA/Vehicles.csv",header=True,inferSchema=True)

# Exploračná analýza

In [4]:
# Funkcia na základnú exploračnú analýzu dát.
# Parametre:
# df -> PySpark DataFrame na  analýzu
# name -> Názov datasetu (len pre výpis)
def eda_summary(df, name="Dataset"):
    print(f"\n{30*'#'} ANALÝZA {name} {30*'#'}\n")
    # Dátové typy
    print("\nDátové typy:\n")
    for column, dtype in df.dtypes:
        print(f"{column}: {dtype}")
    # Prvý záznam (vertikálne)
    print("\nPrvý záznam (vertikálne):\n")
    df.show(n=1, truncate=True, vertical=True)
    # Schéma datasetu
    print("\nSchéma datasetu:\n")
    df.printSchema()
    # Základné štatistiky
    print("\nZákladné štatistiky:\n")
    df.describe().show(vertical=True)
    # Chýbajúce hodnoty
    print("\nChýbajúce hodnoty v datasetu:\n")
    df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show(vertical=True)
    print(f"\n{30*'#'} KONIEC ANALÝZY {name} {30*'#'}\n")

### Accidents
Accident_Index – Unikátny identifikátor nehody.  
Location_Easting_OSGR, Location_Northing_OSGR – Súradnice v britskom súradnicovom systéme (OSGR).  
Longitude, Latitude – Geografická poloha nehody.  
Police_Force – Kód policajnej jednotky, ktorá nehodu riešila.  
Accident_Severity – Závažnosť nehody (napr. mierna, vážna, smrteľná).  
Number_of_Vehicles – Počet vozidiel zapojených do nehody.  
Number_of_Casualties – Počet zranených/osôb postihnutých nehodou.  
Date, Time – Dátum a čas nehody.  
Day_of_Week – Deň v týždni, kedy k nehode došlo.  
Local_Authority_(District), Local_Authority_(Highway) – Miestne samosprávy zodpovedné za danú lokalitu.  
1st_Road_Class, 2nd_Road_Class – Kategória cesty (napr. diaľnica, hlavná cesta).  
1st_Road_Number, 2nd_Road_Number – Číslo cesty (ak existuje).  
Road_Type – Typ cesty (napr. jednosmerná, kruhový objazd).  
Speed_limit – Maximálna povolená rýchlosť v mieste nehody.  
Junction_Detail, Junction_Control – Informácie o križovatke (napr. semafory, kruhový objazd).  
Pedestrian_Crossing-Human_Control, Pedestrian_Crossing-Physical_Facilities – Prítomnosť priechodu pre chodcov a jeho typ.  
Light_Conditions – Svetelné podmienky (napr. denné svetlo, tma).  
Weather_Conditions – Počasie v čase nehody.  
Road_Surface_Conditions – Stav vozovky (suchá, mokrá, zľadovatená).  
Special_Conditions_at_Site – Špeciálne okolnosti (napr. výtlky, olejová škvrna).  
Carriageway_Hazards – Prekážky na ceste (napr. spadnutý strom, stojace vozidlo).  
Urban_or_Rural_Area – Mesto alebo vidiek.  
Did_Police_Officer_Attend_Scene_of_Accident – Či polícia bola na mieste nehody.  
LSOA_of_Accident_Location – Kód malej geografickej oblasti (Lower Super Output Area).  

In [4]:
# eda_summary(accidents,"Accidents Dataset")

### Casualties
Accident_Index – Prepojenie na konkrétnu nehodu.  
Vehicle_Reference – Identifikátor vozidla, v ktorom bola osoba.  
Casualty_Reference – Unikátne ID pre každú obeť v rámci nehody.  
Casualty_Class – Typ účastníka (napr. vodič, spolujazdec, chodec).  
Sex_of_Casualty – Pohlavie zranenej osoby.  
Age_of_Casualty, Age_Band_of_Casualty – Vek obete (presný a vo vekových kategóriách).  
Casualty_Severity – Závažnosť zranenia (mierne, vážne, smrteľné).  
Pedestrian_Location – Kde sa chodec nachádzal (napr. priechod pre chodcov).  
Pedestrian_Movement – Pohyb chodca v čase nehody (napr. bežal, prechádzal cez cestu).  
Car_Passenger – Či bol zranený ako spolujazdec v aute.  
Bus_or_Coach_Passenger – Či bol zranený ako cestujúci v autobuse.  
Pedestrian_Road_Maintenance_Worker – Či bol chodec pracovník údržby ciest.  
Casualty_Type – Typ účastníka nehody (napr. cyklista, motocyklista).  
Casualty_Home_Area_Type – Mestská alebo vidiecka oblasť bydliska obete.

In [5]:
# eda_summary(casualties,"Casualties Dataset")

### Vehicles
Accident_Index – Prepojenie na konkrétnu nehodu.  
Vehicle_Reference – Unikátne ID vozidla v rámci nehody.  
Vehicle_Type – Typ vozidla (napr. osobné auto, nákladné auto, motorka).  
Towing_and_Articulation – Či vozidlo ťahalo príves alebo bolo kĺbové.  
Vehicle_Manoeuvre – Manéver vozidla pred nehodou (napr. odbočovanie, cúvanie).  
Vehicle_Location-Restricted_Lane – Či bolo vozidlo v pruhu s obmedzením (napr. bus pruh).  
Junction_Location – Poloha vozidla na križovatke (napr. hlavná cesta, vedľajšia cesta).  
Skidding_and_Overturning – Či vozidlo dostalo šmyk alebo sa prevrátilo.  
Hit_Object_in_Carriageway – Či narazilo do objektu na ceste (napr. zvodidlá).  
Vehicle_Leaving_Carriageway – Či vozidlo opustilo vozovku.  
Hit_Object_off_Carriageway – Či narazilo do objektu mimo cesty.  
1st_Point_of_Impact – Miesto prvého nárazu na vozidle (napr. predná časť, bočná časť).  
Was_Vehicle_Left_Hand_Drive? – Či malo vozidlo ľavostranné riadenie.  
Journey_Purpose_of_Driver – Účel jazdy (napr. dochádzanie do práce).  
Sex_of_Driver – Pohlavie vodiča.  
Age_of_Driver, Age_Band_of_Driver – Vek vodiča (presný a kategorizovaný).  
Engine_Capacity_(CC) – Objem motora vozidla.  
Propulsion_Code – Typ pohonu (napr. benzín, diesel, elektrický).  
Age_of_Vehicle – Vek vozidla v rokoch.  
Driver_IMD_Decile – Sociálno-ekonomická situácia oblasti, kde vodič býva (Index Multiple Deprivation).  
Driver_Home_Area_Type – Mestská alebo vidiecka oblasť bydliska vodiča.

In [6]:
# eda_summary(vehicles, "Vechicles Dataset")

### Vysporiadanie sa s -1 hodnotami  
-1 hodnoty nahradené priemerom daného atribútu (bez započítania -1 do priemeru)

In [5]:
# Funkcia na zistenie počtu -1 hodnôt
def count_negative_ones(df, df_name):
    # Získanie zoznamu číselných stĺpcov
    numeric_cols = [c.name for c in df.schema.fields if isinstance(c.dataType, NumericType)]
    
    # Výber len číselných stĺpcov na kontrolu -1
    negative_counts = df.select([
        count(when(col(c) == -1, c)).alias(c) for c in numeric_cols
    ])
    
    print(f"\nPočet hodnôt = -1 v {df_name}:\n")
    negative_counts.show(vertical=True)

datasets = {"Accidents": accidents, "Casualties": casualties, "Vehicles": vehicles}

for name, df in datasets.items():
    count_negative_ones(df, name)


Počet hodnôt = -1 v Accidents:

-RECORD 0---------------------------------------------
 Location_Easting_OSGR                       | 0      
 Location_Northing_OSGR                      | 0      
 Longitude                                   | 1      
 Latitude                                    | 0      
 Police_Force                                | 0      
 Accident_Severity                           | 0      
 Number_of_Vehicles                          | 0      
 Number_of_Casualties                        | 0      
 Day_of_Week                                 | 0      
 Local_Authority_(District)                  | 0      
 1st_Road_Class                              | 0      
 1st_Road_Number                             | 2      
 Road_Type                                   | 0      
 Speed_limit                                 | 0      
 Junction_Detail                             | 18     
 Junction_Control                            | 585322 
 2nd_Road_Class                 

In [6]:
# Funcia na nahradenie -1 hodnôt priemerom
def replace_negative_ones(df):
    # Získame len číselné stĺpce
    numeric_cols = [c.name for c in df.schema.fields if isinstance(c.dataType, NumericType)]
    
    # Vypočítame priemery pre každý stĺpec (ignorujeme hodnoty -1)
    mean_values = df.select([mean(when(col(c) != -1, col(c))).alias(c) for c in numeric_cols]).collect()[0]

    # Nahradíme -1 priemernou hodnotou daného stĺpca
    for col_name in numeric_cols:
        avg_value = mean_values[col_name]  # Získame vypočítaný priemer
        if avg_value is not None:  # Skontrolujeme, či priemer existuje (nemôže byť None)
            df = df.withColumn(col_name, when(col(col_name) == -1, avg_value).otherwise(col(col_name)))
    
    return df

# Aplikujeme funkciu na všetky DataFrames
datasets = {
    "Accidents": replace_negative_ones(accidents),
    "Casualties": replace_negative_ones(casualties),
    "Vehicles": replace_negative_ones(vehicles),
}

In [7]:
# Overenie nahradenia hodnôt
for name, df in datasets.items():
    count_negative_ones(df, name)


Počet hodnôt = -1 v Accidents:

-RECORD 0------------------------------------------
 Location_Easting_OSGR                       | 0   
 Location_Northing_OSGR                      | 0   
 Longitude                                   | 0   
 Latitude                                    | 0   
 Police_Force                                | 0   
 Accident_Severity                           | 0   
 Number_of_Vehicles                          | 0   
 Number_of_Casualties                        | 0   
 Day_of_Week                                 | 0   
 Local_Authority_(District)                  | 0   
 1st_Road_Class                              | 0   
 1st_Road_Number                             | 0   
 Road_Type                                   | 0   
 Speed_limit                                 | 0   
 Junction_Detail                             | 0   
 Junction_Control                            | 0   
 2nd_Road_Class                              | 0   
 2nd_Road_Number               

### Sú nejaké atribúty medzi datasetmi zhodné?

In [63]:
# Zistenie názvov atribútov
columns_accidents = set(accidents.columns)
columns_vehicles = set(vehicles.columns)
columns_casualties = set(casualties.columns)

# Zistenie, ktoré atribúty sú spoločné medzi dvoma datasetmi
common_vehicles_casualties = columns_vehicles & columns_casualties - columns_accidents
common_vehicles_accidents = columns_vehicles & columns_accidents - columns_casualties
common_casualties_accidnets = columns_casualties & columns_accidents - columns_vehicles

print("Spoločné iba pre vehicles a casualties:", common_vehicles_casualties)
print("Spoločné iba pre vehicles a accidents:", common_vehicles_accidents)
print("Spoločné iba pre casualties a accidents:", common_casualties_accidnets)

Spoločné iba pre vehicles a casualties: {'Vehicle_Reference'}
Spoločné iba pre vehicles a accidents: set()
Spoločné iba pre casualties a accidents: set()


### Zhodný atribút 'Vehicle_Reference' v datasetoch 'vehicles' a 'casualties'

In [26]:
# Zobrazenie ako často sa vyskytuje n konkrétnych hodnôt v datasete vehicles
# vehicles.groupBy("Vehicle_Reference").count().show(vertical=True,n=5)

In [27]:
# Zobrazenie ako často sa vyskytuje n konkrétnych hodnôt v datasete casualties
# casualties.groupBy("Vehicle_Reference").count().show(vertical=True,n=5)

In [64]:
unique_vehicles = vehicles.select("Vehicle_Reference").subtract(casualties.select("Vehicle_Reference"))
unique_casualties = casualties.select("Vehicle_Reference").subtract(vehicles.select("Vehicle_Reference"))

print("Je iba vo vehicles:")
unique_vehicles.show(vertical=True)

print("Je iba v casualties:")
unique_casualties.show(vertical=True)

Je iba vo vehicles:
-RECORD 0----------------
 Vehicle_Reference | 35  

Je iba v casualties:
(0 rows)



In [65]:
# Je niečo v casualties čo by vo vehicles nebolo (pre atribút Vehicle_Reference)?
# Ak áno, tak výsledok bude False, ak nie je, tak True a teda casualties.Vehicle_Reference je podmnožina vehicles.Vehicle_Reference
print(casualties.select("Vehicle_Reference").subtract(vehicles.select("Vehicle_Reference")).count() == 0)

True


In [66]:
# Záver: Vehicle_Reference vo vehicles obsahuje všetky možné hodnoty Vehicle_Reference z casualties
# Riešenie: Keďže vo vehicles existuje iba jedna hodnota "navyše", odstránime ju
vehicles = vehicles.filter(vehicles.Vehicle_Reference != 35)

In [67]:
# Overenie Vehicle_Reference
unique_vehicles = vehicles.select("Vehicle_Reference").subtract(casualties.select("Vehicle_Reference"))
unique_casualties = casualties.select("Vehicle_Reference").subtract(vehicles.select("Vehicle_Reference"))

print("Je iba vo vehicles:")
unique_vehicles.show(vertical=True)

print("Je iba v casualties:")
unique_casualties.show(vertical=True)

Je iba vo vehicles:
(0 rows)

Je iba v casualties:
(0 rows)



In [68]:
# Použitie aliasov pre oba datasety
vehicles_alias = vehicles.alias("vehicles")
casualties_alias = casualties.alias("casualties")

# Zlúčenie oboch datasetov s aliasami
comparison_df = vehicles_alias.join(casualties_alias, on="Accident_Index", how="left")

# Pridanie nového stĺpca pre porovnanie Vehicle_Reference medzi datasetmi
comparison_df = comparison_df.withColumn(
    "Vehicle_Reference_match",
    (F.col("vehicles.Vehicle_Reference") == F.col("casualties.Vehicle_Reference"))
)

# Zobraziť riadky, kde hodnoty nezhodujú
mismatched_df = comparison_df.filter(F.col("Vehicle_Reference_match") == False)

# Vypísať riadky s rozdielnymi hodnotami
mismatched_df.select("Accident_Index", "vehicles.Vehicle_Reference", "casualties.Vehicle_Reference").show(vertical=True,n=10)

-RECORD 0--------------------------
 Accident_Index    | 200501BS00003 
 Vehicle_Reference | 1             
 Vehicle_Reference | 2             
-RECORD 1--------------------------
 Accident_Index    | 200501BS00006 
 Vehicle_Reference | 1             
 Vehicle_Reference | 2             
-RECORD 2--------------------------
 Accident_Index    | 200501BS00007 
 Vehicle_Reference | 2             
 Vehicle_Reference | 1             
-RECORD 3--------------------------
 Accident_Index    | 200501BS00010 
 Vehicle_Reference | 1             
 Vehicle_Reference | 2             
-RECORD 4--------------------------
 Accident_Index    | 200501BS00010 
 Vehicle_Reference | 2             
 Vehicle_Reference | 1             
-RECORD 5--------------------------
 Accident_Index    | 200501BS00011 
 Vehicle_Reference | 2             
 Vehicle_Reference | 1             
-RECORD 6--------------------------
 Accident_Index    | 200501BS00011 
 Vehicle_Reference | 2             
 Vehicle_Reference | 1      

In [69]:
print(f"Počet rozdielnych záznamov: {mismatched_df.count()}\n\
Počet záznamov vo vehicles: {vehicles.count()}\n\
Počet záznamov v casualties: {casualties.count()}")

Počet rozdielnych záznamov: 2070803
Počet záznamov vo vehicles: 3004424
Počet záznamov v casualties: 2216720


In [70]:
# Treba sa rozodnúť, že z ktorého datasetu vyhodíme atribút Vehicle_Reference
# casualties = casualties.drop("Vehicle_Reference")
vehicles = vehicles.drop("Vehicle_Reference")

### Spojenie datasetov

In [71]:
datasets = {"Accidents": accidents, "Casualties": casualties, "Vehicles": vehicles}

for name, df in datasets.items():
    print(f"Počet atribútov v {name}: {len(df.columns)}")

Počet atribútov v Accidents: 32
Počet atribútov v Casualties: 15
Počet atribútov v Vehicles: 21


In [72]:
merged_df = accidents.join(casualties, on="Accident_Index", how="left") \
                     .join(vehicles, on="Accident_Index", how="left")

print(f"Počet atribútov v zlúčenom DataFrame: {len(merged_df.columns)}")

Počet atribútov v zlúčenom DataFrame: 66


In [73]:
# 32 + 15 + 22 = 69 => Záleží odkiaľ vyhodíme Vehicle_Reference
# 69 - 67 = tri datasety -> tri kľúče (atribút Accident_Index) a po spojení ostal len ako jeden atribút
# 67- 1 (Vehicle_reference z casualties)

### Kde všade (a koľko) sú null hodnoty po odstránení Vehicle_Reference?

In [74]:
null_counts = merged_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in merged_df.columns])
null_counts.show(vertical=True)

-RECORD 0---------------------------------------------
 Accident_Index                              | 0      
 Location_Easting_OSGR                       | 256    
 Location_Northing_OSGR                      | 256    
 Longitude                                   | 256    
 Latitude                                    | 256    
 Police_Force                                | 0      
 Accident_Severity                           | 0      
 Number_of_Vehicles                          | 0      
 Number_of_Casualties                        | 0      
 Date                                        | 0      
 Day_of_Week                                 | 0      
 Time                                        | 264    
 Local_Authority_(District)                  | 0      
 Local_Authority_(Highway)                   | 0      
 1st_Road_Class                              | 0      
 1st_Road_Number                             | 0      
 Road_Type                                   | 0      
 Speed_lim

1. Sampling a Rozdelenie na Tréningovaciu a Testovaciu Množinu
Najprv vytvoríme stratifikovanú vzorku (napr. 10 %) podľa cieľového atribútu. Predpokladáme, že cieľovým atribútom je napr. Accident_Severity (môžete upraviť podľa potreby). Potom vzorku rozdelíme na trénovaciu (60 %) a testovaciu (40 %).

In [17]:
# merged_df.select("Vehicle_Reference", "`Vehicles.Vehicle_Reference`", "`Casualties.Vehicle_Reference`").show(10)

In [18]:
# merged_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in merged_df.columns]).show(vertical=True)
# print(f"\n{30*'#'} KONIEC ANALÝZY {name} {30*'#'}\n")

In [47]:
# Zobrazenie počtu záznamov
# print("Celkový počet záznamov:", merged_df.count())

# Definujte hodnoty cieľového atribútu a nastavené fraction pre stratifikované sampling
# Napríklad, ak sú hodnoty 1, 2, 3 (kódovanie podľa závažnosti nehody)
target_col = "Accident_Severity"
fractions = {row[target_col]: 0.1 for row in merged_df.select(target_col).distinct().collect()}

# Stratifikované vzorkovanie
sample_df = merged_df.sampleBy(target_col, fractions, seed=42)
# print("Počet záznamov vo vzorke (10%):", sample_df.count())

# Rozdelenie vzorky na tréningovú a testovaciu množinu (60/40)
train_df, test_df = sample_df.randomSplit([0.6, 0.4], seed=42)
# print("Trénovacia množina:", train_df.count(), "záznamov")
# print("Testovacia množina:", test_df.count(), "záznamov")


Celkový počet záznamov: 4287593
Počet záznamov vo vzorke (10%): 429243
Tréningová množina: 257491 záznamov
Testovacia množina: 171191 záznamov


In [ ]:
# Celkový počet záznamov: 4287593
# Počet záznamov vo vzorke (10%): 42924
# Trénovacia množina: 257491 záznamov
# Testovacia množina: 171191 záznamov

2. Predspracovanie Dát
a) Transformácia nominálnych atribútov na numerické
Použijeme StringIndexer pre atribúty ako Accident_Severity a iné kategorizované stĺpce. Pre ilustráciu prevedieme cieľový atribút a jeden ďalší atribút (napr. Weather_Conditions).

In [13]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, Bucketizer
from pyspark.ml import Pipeline

# StringIndexery pre kategóriové atribúty
severity_indexer = StringIndexer(inputCol="Accident_Severity", outputCol="Accident_Severity_Index", handleInvalid="keep")
weather_indexer = StringIndexer(inputCol="Weather_Conditions", outputCol="Weather_Conditions_Index", handleInvalid="keep")

# Prípadne môžeme použiť aj OneHotEncoder, ak budeme potrebovať one-hot reprezentáciu
weather_encoder = OneHotEncoder(inputCol="Weather_Conditions_Index", outputCol="Weather_Conditions_Encoded")

# Ako príklad budeme discretizovať aj numerický atribút, napr. 'Speed_limit'
# Definujte prahy pre Bucketizer – tieto hodnoty upravte podľa rozsahu v dátach
speed_splits = [0, 30, 50, 70, 100, float("Inf")]
speed_bucketizer = Bucketizer(splits=speed_splits, inputCol="Speed_limit", outputCol="Speed_limit_Bucket", handleInvalid="keep")

# Vytvorenie zoznamu vstupných atribútov pre klasifikačný model (príklad)
# Zvoľte atribúty, ktoré budú použité ako vstupy (toto je len ilustrácia – prispôsobte podľa analýzy)
feature_cols = ["Number_of_Vehicles", "Number_of_Casualties", "Speed_limit_Bucket", "Weather_Conditions_Index"]

# Pre zostavenie feature vektora
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features", handleInvalid="keep")

# Zostavenie Pipeline pre predspracovanie
preprocessing_pipeline = Pipeline(stages=[
    severity_indexer,
    weather_indexer,
    weather_encoder,
    speed_bucketizer,
    assembler
])
preprocessed_model = preprocessing_pipeline.fit(train_df)
train_preprocessed = preprocessed_model.transform(train_df)
test_preprocessed = preprocessed_model.transform(test_df)

# Overenie výsledkov
train_preprocessed.select("Accident_Severity", "Accident_Severity_Index", "Speed_limit", "Speed_limit_Bucket", "features").show(3, truncate=False)


+-----------------+-----------------------+-----------+------------------+-----------------+
|Accident_Severity|Accident_Severity_Index|Speed_limit|Speed_limit_Bucket|features         |
+-----------------+-----------------------+-----------+------------------+-----------------+
|3                |0.0                    |30         |1.0               |[2.0,1.0,1.0,0.0]|
|2                |1.0                    |30         |1.0               |[2.0,1.0,1.0,0.0]|
|3                |0.0                    |30         |1.0               |[2.0,1.0,1.0,1.0]|
+-----------------+-----------------------+-----------+------------------+-----------------+
only showing top 3 rows



b) Spracovanie chýbajúcich hodnôt
Ukážeme, ako nahradiť chýbajúce hodnoty, pričom hodnoty "-1" považujeme za neplatné. Tu je príklad pre jeden numerický atribút. Môžete tento prístup aplikovať na viacero stĺpcov.

In [14]:
from pyspark.sql.functions import when, avg

# Príklad: Nahradenie neplatných hodnôt v stĺpci 'Speed_limit'
# Najprv prepočítame priemer, pričom vynecháme hodnoty -1
mean_speed = merged_df.filter(merged_df["Speed_limit"] != -1).select(avg("Speed_limit")).first()[0]
print("Priemerná hodnota Speed_limit:", mean_speed)

# Vytvorenie DataFrame, kde nahradíme -1 priemerom
merged_df_clean = merged_df.withColumn("Speed_limit_clean",
                                       when(col("Speed_limit") == -1, mean_speed)
                                       .otherwise(col("Speed_limit")))
merged_df_clean.select("Speed_limit", "Speed_limit_clean").show(3)


Priemerná hodnota Speed_limit: 41.10451831598755
+-----------+-----------------+
|Speed_limit|Speed_limit_clean|
+-----------+-----------------+
|         30|             30.0|
|         30|             30.0|
|         30|             30.0|
+-----------+-----------------+
only showing top 3 rows



Pre kategóriové atribúty by ste mohli zvoliť najčastejšiu hodnotu (mode). Rovnaký prístup potom opakujte pre ostatné relevantné atribúty.

c) Výpočet informačného zisku pre nominálne atribúty
Jednoduchý spôsob je použiť vstavané metódy modelov (napr. Decision Tree), ktoré dokážu poskytnúť dôležitosť atribútov. 
Ak však potrebujete explicitne vypočítať informačný zisk, môžete si vytvoriť vlastnú funkciu. Tu je ilustratívny príklad 
pre jeden atribút:

import math
from pyspark.sql import functions as F

def compute_entropy(df, col_name):
    # Počet celkových záznamov
    total = df.count()
    # Počet výskytov jednotlivých kategórií
    counts = df.groupBy(col_name).count().collect()
    entropy = 0.0
    for row in counts:
        p = row["count"] / total
        entropy -= p * math.log(p, 2)
    return entropy

# Výpočet entropie cieľového atribútu
entropy_target = compute_entropy(train_df, "Accident_Severity")
print("Entropia cieľového atribútu (Accident_Severity):", entropy_target)

# Pre výpočet informačného zisku pre atribút napr. 'Weather_Conditions'
entropy_weather = compute_entropy(train_df, "Weather_Conditions")

# Spočítanie podmienenej entropie
# Rozdelenie podľa 'Weather_Conditions' a výpočet entropie v každej skupine
group_entropy = train_df.groupBy("Weather_Conditions").agg(F.count("*").alias("cnt")).collect()
weighted_entropy = 0.0
total = train_df.count()
for row in group_entropy:
    p = row["cnt"] / total
    sub_df = train_df.filter(train_df["Weather_Conditions"] == row["Weather_Conditions"])
    weighted_entropy += p * compute_entropy(sub_df, "Accident_Severity")

information_gain = entropy_target - weighted_entropy
print("Informačný zisk pre Weather_Conditions:", information_gain)


3. Modelovanie
a) K-means Clustering a Detekcia Anomálií
Najprv vytvoríme K-means model s predspracovanými dátami a následne zistíme anomálie podľa vzdialenosti od centroidu.

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Použijeme pre clustering atribút 'features' vytvorený v pipeline
kmeans = KMeans(featuresCol="features", predictionCol="cluster", k=5, seed=42)
kmeans_model = kmeans.fit(train_preprocessed)

# Pridanie clusterov pre tréningovú množinu
train_clustered = kmeans_model.transform(train_preprocessed)
train_clustered.select("features", "cluster").show(3)

# Vyhodnotenie clusteringu (napr. pomocou silhouette score)
evaluator = ClusteringEvaluator(featuresCol="features", predictionCol="cluster", metricName="silhouette", distanceMeasure="squaredEuclidean")
silhouette = evaluator.evaluate(train_clustered)
print("Silhouette score:", silhouette)

# Detekcia anomálií:
# Vypočítame vzdialenosť každého bodu od príslušného centroidu
centers = kmeans_model.clusterCenters()
def compute_distance(features, center):
    return float(features.squared_distance(center))  # Funkcia squared_distance nie je priamo dostupná; budeme pracovať pomocou operácií

# Pre ilustráciu využijeme User Defined Function (UDF) pre výpočet Euklidovskej vzdialenosti
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
import numpy as np

def euclidean_distance(v, center):
    return float(np.sqrt(np.sum((np.array(v) - np.array(center))**2)))

# UDF: pre každý riadok vypočíta vzdialenosť od príslušného centroidu
def distance_from_center(features, cluster):
    center = centers[int(cluster)]
    return euclidean_distance(features, center)

distance_udf = udf(distance_from_center, DoubleType())
train_clustered = train_clustered.withColumn("distance", distance_udf(col("features"), col("cluster")))
train_clustered.select("cluster", "distance").show(3)

# Označenie anomálií: napr. tie záznamy, ktorých vzdialenosť presahuje prah – tento prah môžete nastaviť napr. ako percentil vzdialeností
distances = train_clustered.select("distance").rdd.flatMap(lambda x: x).collect()
threshold = np.percentile(distances, 95)  # 95. percentil
print("Prahová hodnota pre anomáliu (95. percentil):", threshold)

anomalies = train_clustered.filter(col("distance") > threshold)
print("Počet detekovaných anomálií:", anomalies.count())


b) Klasifikačné Modely
V nasledujúcom bloku vytvoríme viacero klasifikačných modelov pomocou pipeline – Decision Tree, Linear SVM, Naive Bayes a Random Forest. Ako príklad využijeme už vytvorený pipeline pre predspracovanie. Cieľovým atribútom je Accident_Severity_Index.

In [15]:
from pyspark.ml.classification import DecisionTreeClassifier, LinearSVC, NaiveBayes, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Príprava dát už máme v train_preprocessed a test_preprocessed
# Použijeme "features" a "Accident_Severity_Index" ako target

# 1. Decision Tree
dt = DecisionTreeClassifier(featuresCol="features", labelCol="Accident_Severity_Index", seed=42)
dt_model = dt.fit(train_preprocessed)
dt_predictions = dt_model.transform(test_preprocessed)

# 2. Linear SVM
# Upozornenie: LinearSVC vyžaduje, aby cieľové hodnoty boli indexované a v tvare Double
svm = LinearSVC(featuresCol="features", labelCol="Accident_Severity_Index", maxIter=10, regParam=0.1)
svm_model = svm.fit(train_preprocessed)
svm_predictions = svm_model.transform(test_preprocessed)

# 3. Naive Bayes
nb = NaiveBayes(featuresCol="features", labelCol="Accident_Severity_Index", modelType="multinomial")
nb_model = nb.fit(train_preprocessed)
nb_predictions = nb_model.transform(test_preprocessed)

# 4. Random Forest
rf = RandomForestClassifier(featuresCol="features", labelCol="Accident_Severity_Index", numTrees=20, seed=42)
rf_model = rf.fit(train_preprocessed)
rf_predictions = rf_model.transform(test_preprocessed)

# Funkcia na vyhodnotenie modelu
def evaluate_model(predictions, model_name="Model"):
    evaluator_acc = MulticlassClassificationEvaluator(labelCol="Accident_Severity_Index", predictionCol="prediction", metricName="accuracy")
    evaluator_f1 = MulticlassClassificationEvaluator(labelCol="Accident_Severity_Index", predictionCol="prediction", metricName="f1")
    accuracy = evaluator_acc.evaluate(predictions)
    f1 = evaluator_f1.evaluate(predictions)
    print(f"{model_name} - Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}")
    
    # Kontingenčná tabuľka (confusion matrix)
    predictions.groupBy("Accident_Severity_Index", "prediction").count().orderBy("Accident_Severity_Index", "prediction").show()

print("\nVyhodnotenie Decision Tree:")
evaluate_model(dt_predictions, "Decision Tree")

print("\nVyhodnotenie Linear SVM:")
evaluate_model(svm_predictions, "Linear SVM")

print("\nVyhodnotenie Naive Bayes:")
evaluate_model(nb_predictions, "Naive Bayes")

print("\nVyhodnotenie Random Forest:")
evaluate_model(rf_predictions, "Random Forest")


SparkRuntimeException: [USER_RAISED_EXCEPTION] Labels MUST be in {0, 1}, but got 2.0 SQLSTATE: P0001

4. Výpočet Ďalších Metrík
Pre MCC (Matthews Correlation Coefficient) a ďalšie metriky je možné vypočítať manuálne na základe confusion matrix. Tu je jednoduchý príklad pre binárny prípad; pre viactriedne problémy by bolo potrebné rozšíriť výpočet.

In [16]:
import numpy as np
import pandas as pd

# Funkcia pre výpočet MCC z confusion matrix pre binárny prípad
def compute_mcc(confusion_pd):
    # Predpokladáme, že confusion_pd má stĺpce: label, prediction, count
    # Najprv extrahujeme TP, TN, FP, FN – toto funguje len pre binárnu klasifikáciu
    # Upravte podľa potreby pre viac tried
    tp = confusion_pd[(confusion_pd['label'] == 1) & (confusion_pd['prediction'] == 1)]['count'].sum()
    tn = confusion_pd[(confusion_pd['label'] == 0) & (confusion_pd['prediction'] == 0)]['count'].sum()
    fp = confusion_pd[(confusion_pd['label'] == 0) & (confusion_pd['prediction'] == 1)]['count'].sum()
    fn = confusion_pd[(confusion_pd['label'] == 1) & (confusion_pd['prediction'] == 0)]['count'].sum()
    numerator = tp * tn - fp * fn
    denominator = np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    if denominator == 0:
        return 0
    return numerator/denominator

# Príklad pre Decision Tree (ak máte binárnu klasifikáciu)
dt_conf_df = dt_predictions.groupBy("Accident_Severity_Index", "prediction").count().toPandas()
print("Confusion matrix (Decision Tree):")
print(dt_conf_df)
# Ak je to viactriedna klasifikácia, MCC sa zvyčajne počíta inak.
# print("MCC (Decision Tree):", compute_mcc(dt_conf_df))


Confusion matrix (Decision Tree):
   Accident_Severity_Index  prediction   count
0                      2.0         0.0    3162
1                      1.0         1.0     413
2                      0.0         1.0      84
3                      1.0         0.0   23593
4                      2.0         1.0      73
5                      0.0         0.0  143790
6                      0.0         2.0       6
7                      2.0         2.0      70


Zhrnutie
Tento návrh kódu rozširuje Váš pôvodný .ipynb súbor o:

Stratifikované sampling a rozdelenie dát.

Predspracovanie (transformácia kategórií, discretizácia numerických atribútov, nahradenie chýbajúcich hodnôt).

Výpočet informačného zisku pre vybraný atribút.

Vytvorenie K-means modelu a jednoduchú detekciu anomálií.

Viaceré klasifikačné modely (Decision Tree, Linear SVM, Naive Bayes, Random Forest) a ich vyhodnotenie pomocou confusion matrix a základných metrík.

Ak by ste chceli prebrať konkrétnu časť podrobnejšie alebo upraviť niektoré kroky, dajte vedieť, rád pomôžem ďalej.